In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# deletion
def indicineprotvar(hfprotvar_wd, chr, outloc):
  print("hffinal_df", hfprotvar_wd.shape)
  in_f1=f"/content/drive/MyDrive/gene_ext/ind_protein/ind_chr{chr}_prot.csv"
  ind_prot=pd.read_csv(in_f1)
  ind_prot["indstart_of_next_gene"] = ind_prot["Start"].shift(-1).fillna(0)
  finalindprot_df=pd.DataFrame(columns=list(hfprotvar_wd)+["NelStart"]+["NelStop"]+["NelLocus"]+["NelLength"]+["NelProt"]+["Nelnextgen"]+["Nelgene_type"])
  for i, row in tqdm(ind_prot.iterrows()):
    # Check for the at start
    if hfprotvar_wd[(row["Start"]==hfprotvar_wd["startpos.q"]) & (row["Stop"]>hfprotvar_wd["endpos.q"])].shape[0]>=1:
        tmp_df = hfprotvar_wd[(row["Start"]==hfprotvar_wd["startpos.q"]) & (row["Stop"]>hfprotvar_wd["endpos.q"])].copy(deep=True)
        tmp_df["Nelgene_type"] = "at_start"
        tmp_df["NelStart"] = row["Start"]
        tmp_df["NelStop"] = row["Stop"]
        tmp_df["NelLocus"] = row["Locus"]
        tmp_df["NelLength"] = row["Length"]
        tmp_df["NelProt"] = row["Protein Name"]
        tmp_df["Nelnextgen"] = row["indstart_of_next_gene"]
        finalindprot_df = pd.concat([finalindprot_df,tmp_df]).reset_index(drop=True)
    # check for the at end
    elif hfprotvar_wd[(row["Start"]<hfprotvar_wd["startpos.q"]) & (row["Stop"]==hfprotvar_wd["endpos.q"])].shape[0]>=1:
        tmp_df = hfprotvar_wd[(row["Start"]<hfprotvar_wd["startpos.q"]) & (row["Stop"]==hfprotvar_wd["endpos.q"])].copy(deep=True)
        tmp_df["Nelgene_type"] = "at_end"
        tmp_df["NelStart"] = row["Start"]
        tmp_df["NelStop"] = row["Stop"]
        tmp_df["NelLocus"] = row["Locus"]
        tmp_df["NelLength"] = row["Length"]
        tmp_df["NelProt"] = row["Protein Name"]
        tmp_df["Nelnextgen"] = row["indstart_of_next_gene"]
        finalindprot_df = pd.concat([finalindprot_df,tmp_df]).reset_index(drop=True)
    # Check for the within gene
    elif hfprotvar_wd[(row["Start"]<hfprotvar_wd["startpos.q"]) & (row["Stop"]>hfprotvar_wd["endpos.q"])].shape[0]>=1:
        tmp_df = hfprotvar_wd[(row["Start"]<hfprotvar_wd["startpos.q"]) & (row["Stop"]>hfprotvar_wd["endpos.q"])].copy(deep=True)
        tmp_df["Nelgene_type"] = "within_gene"
        tmp_df["NelStart"] = row["Start"]
        tmp_df["NelStop"] = row["Stop"]
        tmp_df["NelLocus"] = row["Locus"]
        tmp_df["NelLength"] = row["Length"]
        tmp_df["NelProt"] = row["Protein Name"]
        tmp_df["Nelnextgen"] = row["indstart_of_next_gene"]
        finalindprot_df = pd.concat([finalindprot_df,tmp_df]).reset_index(drop=True)
    # Check for the intergenic gene
    elif hfprotvar_wd[(row["Stop"]<hfprotvar_wd["startpos.q"]) & (row["indstart_of_next_gene"]>hfprotvar_wd["endpos.q"])].shape[0]>=1:
        tmp_df = hfprotvar_wd[(row["Stop"]<hfprotvar_wd["startpos.q"]) & (row["indstart_of_next_gene"]>hfprotvar_wd["endpos.q"])].copy(deep=True)
        tmp_df["Nelgene_type"] = "intergenic_gene"
        tmp_df["NelStart"] = row["Start"]
        tmp_df["NelStop"] = row["Stop"]
        tmp_df["NelLocus"] = row["Locus"]
        tmp_df["NelLength"] = row["Length"]
        tmp_df["NelProt"] = row["Protein Name"]
        tmp_df["Nelnextgen"] = row["indstart_of_next_gene"]
        finalindprot_df = pd.concat([finalindprot_df,tmp_df]).reset_index(drop=True)
    else:
        pass


  # Generate output file
  out_file16=f"hfqtl_NelVar{chr}.csv"
  finalindprot_df.to_csv(out_file16)
  shutil.copy(out_file16, outloc)

  # # extract only the immune genes
  # hfimm_nel = pd.DataFrame
  # hfimm_nel=finalindprot_df[finalindprot_df["keyword_found"]==True]
  # out_file17=f"hfimm_nel{chr}.csv"
  # hfimm_nel.to_csv(out_file17)
  # shutil.copy(out_file17, outloc)

  # LOV50
  hfNel_LOV50=finalindprot_df[finalindprot_df["LOV"]>=50]
  # hfNelimm_LOV50=hfimm_nel[hfimm_nel["LOV"]>=50]
  out_file18=f"hfNel_LOV50_{chr}.csv"
  # out_file19=f"hfNelimm_LOV50_{chr}.csv"
  hfNel_LOV50.to_csv(out_file18)
  # hfNelimm_LOV50.to_csv(out_file19)
  shutil.copy(out_file18, outloc)
  # shutil.copy(out_file19, outloc)


In [ ]:
def hfqtl_VarAnal(common_df, hfqtl_df, chr, outloc):
  # Identifying the variations
  final_df = pd.DataFrame(columns=list(hfqtl_df.columns)+["HFgene_type"]+["startpos.r"]+["endpos.r"]+["startpos.q"]+["endpos.q"]+["seq.r"]+["seq.q"]+["LOV"])
  for i, row in tqdm(common_df.iterrows()):
    # Check for the at start
    if hfqtl_df[(hfqtl_df["Begin"]==row["startpos.r"]) & (hfqtl_df["End"]>row["endpos.r"])].shape[0]>=1:
        tmp_df = hfqtl_df[(hfqtl_df["Begin"]==row["startpos.r"]) & (hfqtl_df["End"]>row["endpos.r"])].copy(deep=True)
        tmp_df["HFgene_type"] = "at_start"
        tmp_df["startpos.r"] = row["startpos.r"]
        tmp_df["endpos.r"] = row["endpos.r"]
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
    # check for the at end
    elif hfqtl_df[(hfqtl_df["Begin"]<row["startpos.r"]) & (hfqtl_df["End"]==row["endpos.r"])].shape[0]>=1:
        tmp_df = hfqtl_df[(hfqtl_df["Begin"]<row["startpos.r"]) & (hfqtl_df["End"]==row["endpos.r"])].copy(deep=True)
        tmp_df["HFgene_type"] = "at_end"
        tmp_df["startpos.r"] = row["startpos.r"]
        tmp_df["endpos.r"] = row["endpos.r"]
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
    # Check for the within gene
    elif hfqtl_df[(hfqtl_df["Begin"]<row["startpos.r"]) & (hfqtl_df["End"]>row["endpos.r"])].shape[0]>=1:
        tmp_df = hfqtl_df[(hfqtl_df["Begin"]<row["startpos.r"]) & (hfqtl_df["End"]>row["endpos.r"])].copy(deep=True)
        tmp_df["HFgene_type"] = "within_gene"
        tmp_df["startpos.r"] = row["startpos.r"]
        tmp_df["endpos.r"] = row["endpos.r"]
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
    # Check for the intergenic gene
    elif hfqtl_df[(hfqtl_df["End"]<row["startpos.r"]) & (hfqtl_df["hfstart_of_next_gene"]>row["endpos.r"])].shape[0]>=1:
        tmp_df = hfqtl_df[(hfqtl_df["End"]<row["startpos.r"]) & (hfqtl_df["hfstart_of_next_gene"]>row["endpos.r"])].copy(deep=True)
        tmp_df["HFgene_type"] = "intergenic_gene"
        tmp_df["startpos.r"] = row["startpos.r"]
        tmp_df["endpos.r"] = row["endpos.r"]
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
    # Check for the at start with intergenic gene
    elif hfqtl_df[(hfqtl_df["Begin"]>row["startpos.r"]) & (hfqtl_df["Begin"]==row["endpos.r"])].shape[0]>=1:
        tmp_df = hfqtl_df[(hfqtl_df["Begin"]>row["startpos.r"]) & (hfqtl_df["Begin"]==row["endpos.r"])].copy(deep=True)
        tmp_df["HFgene_type"] = "intergenic_gene"
        tmp_df["startpos.r"] = row["startpos.r"]
        tmp_df["endpos.r"] = row["endpos.r"]
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
    # Check for the at end with intergenic gene
    elif hfqtl_df[(hfqtl_df["End"]==row["startpos.r"]) & (hfqtl_df["End"]<row["endpos.r"])].shape[0]>=1:
        tmp_df = hfqtl_df[(hfqtl_df["End"]==row["startpos.r"]) & (hfqtl_df["End"]<row["endpos.r"])].copy(deep=True)
        tmp_df["HFgene_type"] = "intergenic_gene"
        tmp_df["startpos.r"] = row["startpos.r"]
        tmp_df["endpos.r"] = row["endpos.r"]
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
    # Check for the near start with intergenic gene
    elif hfqtl_df[(hfqtl_df["Begin"]>row["startpos.r"]) & (hfqtl_df["Begin"]<row["endpos.r"])].shape[0]>=1:
        tmp_df = hfqtl_df[(hfqtl_df["Begin"]>row["startpos.r"]) & (hfqtl_df["Begin"]<row["endpos.r"])].copy(deep=True)
        tmp_df["HFgene_type"] = "intergenic_gene"
        tmp_df["startpos.r"] = row["startpos.r"]
        tmp_df["endpos.r"] = row["endpos.r"]
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
    # Check for the near end with intergenic gene
    elif hfqtl_df[(hfqtl_df["End"]>row["startpos.r"]) & (hfqtl_df["End"]<row["endpos.r"])].shape[0]>=1:
        tmp_df = hfqtl_df[(hfqtl_df["End"]>row["startpos.r"]) & (hfqtl_df["End"]<row["endpos.r"])].copy(deep=True)
        tmp_df["HFgene_type"] = "intergenic_gene"
        tmp_df["startpos.r"] = row["startpos.r"]
        tmp_df["endpos.r"] = row["endpos.r"]
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
    else:
        pass

  # Generate output file
  out_file3=f"hfprotvar{chr}.csv"
  final_df.to_csv(out_file3)
  shutil.copy(out_file3, outloc)

  # find variation not present in gene and intergene region called as other non coding region
  final_df1 = pd.DataFrame(columns=common_df.columns)
  for index, row in tqdm(common_df.iterrows()):
    # Check if 'startpos.q' is present in any row of file2
    if row['startpos.q'] not in final_df['startpos.q'].values:
        # Append the row to the DataFrame if not found
        final_df1 = final_df1.append(row, ignore_index=True)

  # Save the DataFrame with rows not found in file2 to a new CSV file
  out_file4=f"non_coding{chr}.csv"
  final_df1.to_csv(out_file4, index=False)
  shutil.copy(out_file4, outloc)

  # remove duplicates
  hfprotvar_wd = final_df.drop_duplicates(subset=['Symbol', "QTL Class", 'startpos.r', 'endpos.r'])
  out_file5=f"hfprotvar_wd{chr}.csv"
  hfprotvar_wd.to_csv(out_file5)
  shutil.copy(out_file5, outloc)

  # extract only the immune genes
  hfimm = pd.DataFrame
  hfimm=hfprotvar_wd[hfprotvar_wd["keyword_found"]==True]
  out_file6=f"hfimm{chr}.csv"
  hfimm.to_csv(out_file6)
  shutil.copy(out_file6, outloc)

  # Extract the variation presents in gene and intergene seperately
  hf_within_gene=hfprotvar_wd[hfprotvar_wd["HFgene_type"]=="within_gene"]
  hf_inter_gene=hfprotvar_wd[hfprotvar_wd["HFgene_type"]=="intergenic_gene"]

  # extract the variations among genomic regions including gene and intergene has greater the 50 LOV (length of varaition)
  hf_LOV50=hfprotvar_wd[hfprotvar_wd["LOV"]>=50]

  out_file7=f"hf_within_gene{chr}.csv"
  out_file8=f"hf_inter_gene{chr}.csv"
  out_file9=f"hf_LOV50_{chr}.csv"

  hf_within_gene.to_csv(out_file7)
  hf_inter_gene.to_csv(out_file8)
  hf_LOV50.to_csv(out_file9)

  shutil.copy(out_file7, outloc)
  shutil.copy(out_file8, outloc)
  shutil.copy(out_file9, outloc)

  # Extract the variation presents in immune gene and intergene seperately
  hfimm_within_gene=hfimm[hfimm["HFgene_type"]=="within_gene"]
  hfimm_inter_gene=hfimm[hfimm["HFgene_type"]=="intergenic_gene"]

  # extract the variations among genomic regions including gene and intergene has greater the 50 LOV (length of varaition)
  hfimm_LOV50=hfimm[hfimm["LOV"]>=50]

  out_file10=f"hfimm_within_gene{chr}.csv"
  out_file11=f"hfimm_inter_gene{chr}.csv"
  out_file12=f"hfimm_LOV50_{chr}.csv"

  hfimm_within_gene.to_csv(out_file10)
  hfimm_inter_gene.to_csv(out_file11)
  hfimm_LOV50.to_csv(out_file12)

  shutil.copy(out_file10, outloc)
  shutil.copy(out_file11, outloc)
  shutil.copy(out_file12, outloc)

  # count the variation based on gene type
  hfvar_counts = hfprotvar_wd.drop_duplicates(subset=['Symbol', 'startpos.r', 'endpos.r'])
  gene_count=hfvar_counts.groupby(["HFgene_type"]).size().reset_index(name="count")
  out_file13=f"var_count{chr}.csv"
  gene_count.to_csv(out_file13)
  shutil.copy(out_file13, outloc)

  # extract the immune has how many variations and its gene type
  hfimmvar_counts = hfimm.drop_duplicates(subset=['Symbol', 'startpos.r', 'endpos.r'])
  imm_count=hfimmvar_counts.groupby(by=["Symbol","Name", "HFgene_type"]).size().reset_index(name="count")
  out_file14=f"immlocus_gene_type_counts{chr}.csv"
  imm_count.to_csv(out_file14, index=False)
  shutil.copy(out_file14, outloc)

  # Calculate value counts
  count = hfprotvar_wd["HFgene_type"].value_counts().reset_index()
  count.columns = ["HFgene_type", "Count"]
  count.loc[len(count.index)]=["immune_gene", len(hfimm_within_gene)]
  count.loc[len(count.index)]=["non_coding_gene", len(final_df1)]


  # Save the result to a CSV file
  out_file15=f"delcount{chr}.csv"
  count.to_csv(out_file15, index=False)
  shutil.copy(out_file15, outloc)

  #extract idicine genes
  indicineprotvar(hfprotvar_wd, chr, outloc)

In [ ]:
def map_keywords(str_sentence):
    list_keywords = [
        "immunoglobulin","immunoreceptor","autoimmune","TLR","IgG",
        "autoimmune","autophagy","immunogen","immune","innate","T-cell","NF-kappa", "antigen",
        "B-cell","lymphocyte","histocompatibility","CD24","CD4","LY96", "BCR",
        "IFIT3","PGLYRP1","NKG2D","UL16","leukocyte","cytokine", "interleukin","interferon"
        "antimicrobial peptide","beta-defensin 2","IL16","IL2","chemokine", "antibody"
    ]
    bool_found = bool(
        [
            i for i in list_keywords if i.lower() in str_sentence.lower()
        ]
    )
    return bool_found




import pandas as pd
from tqdm import tqdm
import shutil

Chr_list=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, "X"]
# Chr_list=["X"]
for chr in Chr_list:
  in_file1=f"/content/drive/MyDrive/gene_ext/del/{chr}/common_wd{chr}.csv"
  in_file2=f"/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_qtl/{chr}/HF_QTL{chr}.csv"
  outloc=f"/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/{chr}"
  common_df=pd.read_csv(in_file1)
  hfqtl_df=pd.read_csv(in_file2)

  print("common_df", common_df.shape)
  print("hfqtl_df", hfqtl_df.shape)


  common_df["LOV"]=abs(common_df["endpos.r"]-common_df["startpos.r"])
  out_f1=f"common{chr}_LOV.csv"

  common_df.to_csv(out_f1)
  shutil.copy(out_f1, outloc)

  hfqtl_df["keyword_found"] = hfqtl_df.apply(lambda row: map_keywords(row["Name"]),axis=1)
  out_f2=f"hfqtlprot{chr}.csv"

  hfqtl_df.to_csv(out_f2)
  shutil.copy(out_f2, outloc)

  hfqtl_VarAnal(common_df, hfqtl_df, chr, outloc)

common_df (1374, 13)
hfqtl_df (3435, 19)


1374it [00:11, 118.33it/s]
1374it [00:00, 4813.43it/s]


hffinal_df (3889, 28)


1433it [00:06, 212.43it/s]


common_df (1041, 13)
hfqtl_df (3898, 19)


...

970it [00:03, 264.34it/s]


common_df (1913, 13)
hfqtl_df (9880, 19)


1913it [00:14, 133.96it/s]
1913it [00:00, 6196.88it/s]


hffinal_df (2809, 28)


993it [00:04, 230.00it/s]


In [ ]:
# QTL Analysis - Hereford : Nelore

import pandas as pd
import shutil

chr=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,"X"]
# chr=[1]
for i in chr:
  print(f"Chromosome{i}")

  # set the file path
  inp_f1=f'/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/{i}/hfqtl_NelVar{i}.csv'
  outp_loc=f"/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/{i}"

  # read the file
  dff=pd.read_csv(inp_f1)
  # print(in_f1)
  print("file:", dff.shape)
  # print(df)

  # filter the LOV has greater than equal to 10
  filter_lov10=dff[dff["LOV"]>=10]

  # ignore the qtl class has nil value
  filter_qtl=filter_lov10[filter_lov10["QTL Class"]!="Nil"]
  print("filter_qtl:", filter_qtl.shape)

  # filter the qtlgenetype has within gene type
  filter_qtl=filter_qtl[filter_qtl["qtlgene_type"]=="within_gene"]
  filter_qtl.shape
  print("filter_qtl:", filter_qtl.shape)

  # filter the HFgenetype has within gene type
  filter_hf=filter_qtl[filter_qtl["HFgene_type"]=="within_gene"]
  filter_hf.shape
  print("filte_hf:", filter_hf.shape)

  # filter the Nelgenetype has within gene type
  filter_nel=filter_hf[filter_hf["Nelgene_type"]=="within_gene"]
  filter_nel.shape
  print("filter_nel:", filter_nel.shape)

  filter_nel=filter_nel.drop_duplicates(subset=["Symbol", "QTL Class", "startpos.q", "endpos.q", "LOV"]).reset_index()
  filter_nelq=filter_nel.groupby(["Name", "Symbol", "NelLocus", "NelProt", "startpos.q", "endpos.q", "LOV"]).agg({"QTL Class": lambda x: ",".join(set(x))}).reset_index()

  # Group by 'Name' and 'Symbol' and aggregate the sum of 'LOV' and join 'QTL Class'
  del_qtlHN=filter_nelq.groupby(["Symbol", "Name", "NelLocus", "NelProt", "LOV"]).agg({"QTL Class": ",".join}).reset_index()

  # Drop duplicate values in the 'QTL Class' column
  del_qtlHN['QTL Class'] = del_qtlHN['QTL Class'].apply(lambda x: ', '.join(pd.unique(x.split(','))))

  # save the file into destination
  out_f1=f"del_qtlHN{i}.csv"
  del_qtlHN.to_csv(out_f1)
  print("del_qtlHN:", del_qtlHN.shape)
  shutil.copy(out_f1, outp_loc)

  # Save the file into destination
  del_qtlHN50=del_qtlHN[del_qtlHN["LOV"]>=50]
  out_f2=f"del_qtl50HN{i}.csv"
  del_qtlHN50.to_csv(out_f2)
  print("del_qtlHN50:", del_qtlHN50.shape)
  shutil.copy(out_f2, outp_loc)

Chromosome1
file: (3433, 36)
filter_qtl: (80, 36)
filter_qtl: (36, 36)
filte_hf: (34, 36)
filter_nel: (34, 36)
del_qtlHN: (6, 6)
del_qtlHN50: (0, 6)
Chromosome2
file: (2580, 36)
filter_qtl: (111, 36)
filter_qtl: (82, 36)
filte_hf: (81, 36)
filter_nel: (75, 36)
del_qtlHN: (9, 6)
del_qtlHN50: (2, 6)
Chromosome3
file: (2346, 36)
filter_qtl: (60, 36)
filter_qtl: (28, 36)
filte_hf: (27, 36)
filter_nel: (26, 36)
del_qtlHN: (11, 6)
del_qtlHN50: (0, 6)
Chromosome4
file: (3622, 36)
filter_qtl: (85, 36)
filter_qtl: (56, 36)
filte_hf: (55, 36)
filter_nel: (34, 36)
del_qtlHN: (12, 6)
del_qtlHN50: (1, 6)
Chromosome5
file: (1728, 36)
filter_qtl: (34, 36)
filter_qtl: (28, 36)
filte_hf: (26, 36)
filter_nel: (15, 36)
del_qtlHN: (3, 6)
del_qtlHN50: (0, 6)
Chromosome6
file: (3873, 36)
filter_qtl: (128, 36)
filter_qtl: (106, 36)
filte_hf: (93, 36)
filter_nel: (85, 36)
del_qtlHN: (14, 6)
del_qtlHN50: (1, 6)
Chromosome7
file: (3040, 36)
filter_qtl: (40, 36)
filter_qtl: (22, 36)
filte_hf: (19, 36)
filter_nel

In [ ]:
# Table preparation

# Table preparations

f1=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/1/del_qtl50HN1.csv")
f2=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/2/del_qtl50HN2.csv")
f3=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/3/del_qtl50HN3.csv")
f4=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/4/del_qtl50HN4.csv")
f5=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/5/del_qtl50HN5.csv")
f6=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/6/del_qtl50HN6.csv")
f7=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/7/del_qtl50HN7.csv")
f8=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/8/del_qtl50HN8.csv")
f9=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/9/del_qtl50HN9.csv")
f10=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/10/del_qtl50HN10.csv")
f11=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/11/del_qtl50HN11.csv")
f12=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/12/del_qtl50HN12.csv")
f13=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/13/del_qtl50HN13.csv")
f14=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/14/del_qtl50HN14.csv")
f15=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/15/del_qtl50HN15.csv")
f16=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/16/del_qtl50HN16.csv")
f17=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/17/del_qtl50HN17.csv")
f18=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/18/del_qtl50HN18.csv")
f19=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/19/del_qtl50HN19.csv")
f20=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/20/del_qtl50HN20.csv")
f21=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/21/del_qtl50HN21.csv")
f22=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/22/del_qtl50HN22.csv")
f23=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/23/del_qtl50HN23.csv")
f24=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/24/del_qtl50HN24.csv")
f25=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/25/del_qtl50HN25.csv")
f26=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/26/del_qtl50HN26.csv")
f27=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/27/del_qtl50HN27.csv")
f28=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/28/del_qtl50HN28.csv")
f29=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/29/del_qtl50HN29.csv")
fX=pd.read_csv("/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/del/X/del_qtl50HNX.csv")

In [ ]:
table_delqtl50HN=pd.DataFrame(columns=["Chromosome number"]+list(f1))

In [ ]:
f1["Chromosome number"]=1
f2["Chromosome number"]=2
f3["Chromosome number"]=3
f4["Chromosome number"]=4
f5["Chromosome number"]=5
f6["Chromosome number"]=6
f7["Chromosome number"]=7
f8["Chromosome number"]=8
f9["Chromosome number"]=9
f10["Chromosome number"]=10
f11["Chromosome number"]=11
f12["Chromosome number"]=12
f13["Chromosome number"]=13
f14["Chromosome number"]=14
f15["Chromosome number"]=15
f16["Chromosome number"]=16
f17["Chromosome number"]=17
f18["Chromosome number"]=18
f19["Chromosome number"]=19
f20["Chromosome number"]=20
f21["Chromosome number"]=21
f22["Chromosome number"]=22
f23["Chromosome number"]=23
f24["Chromosome number"]=24
f25["Chromosome number"]=25
f26["Chromosome number"]=26
f27["Chromosome number"]=27
f28["Chromosome number"]=28
f29["Chromosome number"]=29
fX["Chromosome number"]="X"

In [ ]:
table_delqtl50HN=pd.concat([table_delqtl50HN, f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12, f13, f14, f15, f16, f17, f18, f19, f20, f21, f22, f23, f24, f25, f26, f27, f28, f29, fX])


In [ ]:
table_delqtl50HN.shape

(22, 8)

In [ ]:
table_delqtl50HN.to_csv("table_delqtl50HN.csv")
shutil.copy("table_delqtl50HN.csv", "/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar")

'/content/drive/MyDrive/Hereford_VarAnalysis/Hereford_chrVar/table_delqtl50HN.csv'